# Transaction IDS

Every transaction is identified by a unique `VirtualTransactionId` (`vxid`), which is comprised of a backend ID (or `backendID`) and a sequentially-assigned number local to each backend, known as `localXID`.

Non-virtual TransactionIds (`xid`), are assigned sequentially to transactions from a global counter used by all databases within the PostgreSQL cluster. This assignment happens when a transaction *first writes* to the database.

```SQL
SELECT pg_current_xact_id_if_assigned(); -- Nothing, because there is no xid

-- Get vxid of current transaction
SELECT 
	pid,
	relation::regclass AS relation,
	virtualxid as vxid,
	transactionid as xid,
	mode
FROM pg_locks
WHERE pid = (SELECT pg_backend_pid());
```

```SQL
BEGIN; 
DROP TABLE IF EXISTS A; 
CREATE TABLE A(A INT); 
SELECT pg_current_xact_id_if_assigned(); -- Got an XID
ROLLBACK;
```

txids can be compared with each other (in a circle):

<img src="./helpers/txid-comparison.png" alt="drawing" width="900"/>

## Transaction ID Wraparound

Since transaction IDs have limited size (32 bits) a cluster that runs for a long time (more than 4 billion transactions) would suffer transaction ID wraparound: the XID counter wraps around to zero, and all of a sudden transactions that were in the past appear to be in the future — which means their output become invisible.

To prevent this, PostgreSQL reserves a special XID, `FrozenTransactionId`, which does not follow the normal XID comparison rules and is always considered older than every normal XID. Frozen row versions are treated as if the inserting XID were `FrozenTransactionId`, so that they will appear to be “in the past” to all normal transactions regardless of wraparound issues, and so such row versions will be valid until deleted, no matter how long that is.